In [1]:
import json
import re
import numpy as np
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
# import nltk
# nltk.download()

In [3]:
x=[]
y=[]
with open('Sarcasm_Headlines_Dataset.json', 'r') as f:
    i=0
    for line in f:
        json_line = json.loads(line)
        x.append(json_line["headline"])
        y.append(json_line["is_sarcastic"])
        i=i+1

In [4]:
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english')) 

In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
sno = nltk.stem.SnowballStemmer('english')

In [21]:
words = nltk.word_tokenize("leaves")
words = [wordnet_lemmatizer.lemmatize(word, pos="n") for word in words]
[sno.stem(word) for word in words]

['leaf']

In [6]:
for i in range(len(x)):
    x[i] = nltk.word_tokenize(x[i])
    punctuations="?:!.,;"
    for word in x[i]:
        if word in punctuations:
            x[i].remove(word)
    x[i] = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in x[i]]

In [7]:
# for i in range(len(x)):
#    #x[i]=re.split("(\W)",x[i].lower())
#     x[i]=re.split("(\s)",x[i].lower())
#     x[i] = [a for a in x[i] if a != ' ']
#     x[i] = [a for a in x[i] if a != '']
#     x[i] = [a for a in x[i] if a != ',']
#     x[i] = [a for a in x[i] if a != '.']
#     x[i] = [a for a in x[i] if a != '-']
#     #x[i] = [a for a in x[i] if a != "'"]
#     #x[i] = [a for a in x[i] if a not in stop_words]

In [8]:
from sklearn.utils import shuffle
x, y = shuffle(x, y, random_state=0)
x[1:3]

[['classmates.com',
  'employees',
  'do',
  "n't",
  'have',
  'heart',
  'to',
  'tell',
  'ceo',
  'about',
  'facebook'],
 ['woman',
  'unaware',
  'she',
  "'s",
  'only',
  'person',
  'on',
  'acid',
  'at',
  'jam',
  'taylor',
  'concert']]

In [9]:
y = np.array(y)
length = len(sorted(x,key=len, reverse=True)[0])
x = np.array([xi+['<end>']+['<pad>']*(length-len(xi)) for xi in x])
X_train = x[:3500]
Y_train = y[:3500]
X_test = x[4000:4500]
Y_test = y[4000:4500]
X_val = x[3500:4000]
Y_val = y[3500:4000]

In [10]:
X_train = np.fliplr(X_train)
X_test = np.fliplr(X_test)
X_val = np.fliplr(X_val)

In [11]:
word_list = []
word_list.append('<pad>')

In [12]:
for i in X_train[:2000]:
    for j in i:
        if j not in word_list:
            word_list.append(j)

In [13]:
word_list.append('<end>')
word_list.append('<unk>')
num_words = len(word_list)
print(num_words)
print(len(set(word_list)))

5739
5738


In [14]:
def get_index(s):
    if s=='<pad>':
        return 0
    ind = -1
    try:
        ind = word_list.index(s)
    except:
        ind = num_words-1
    return ind

In [15]:
X_train, Y_train = shuffle(X_train, Y_train, random_state=0)
print(X_train.shape)
print(Y_train.shape)

(3500, 42)
(3500,)


In [16]:
X_train[0]

array(['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<end>', 'again',
       'run', 'to', 'plan', 'ahmadinejad', 'mahmoud', 'president',
       'ex-iranian'], dtype='<U56')

In [17]:
def to_oh(Xi):
    return np.array([np.array([get_index(s) for s in a]) for a in Xi])

In [18]:
X_train = to_oh(X_train)
X_test = to_oh(X_test)
X_val = to_oh(X_val)

In [19]:
from keras.utils import to_categorical

In [20]:
X_train_oh = np.array(list(map(lambda x1: to_categorical(x1, num_classes=num_words), X_train)))
X_test_oh = np.array(list(map(lambda x1: to_categorical(x1, num_classes=num_words), X_test)))
X_val_oh = np.array(list(map(lambda x1: to_categorical(x1, num_classes=num_words), X_val)))

In [21]:
print(X_train_oh.shape)
print(Y_train.shape)

(3500, 42, 5739)
(3500,)


In [22]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          1, 1507,  602,   19,  712, 4332, 4333,  183, 4334])

model = keras.models.Sequential()
model.add(keras.layers.LSTM(X_train.shape[1]))
#model.add(keras.layers.LSTM(X_train.shape[1]))
model.add(keras.layers.Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01), metrics=['accuracy'])
#print(model.summary())
model.fit(X_train_oh, Y_train, epochs=8, batch_size=128)

scores = model.evaluate(X_test_oh, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [23]:
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, Concatenate
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.regularizers import l1,l2

In [ ]:
reshapor = Reshape((1, X_train_oh.shape[2]))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(X_train_oh.shape[1], return_state = True)         # Used in Step 2.C
densor = Dense(1, activation='sigmoid') # Used in Step 2.D
dropout = Dropout(0.5)
X = Input(shape=(X_train_oh.shape[1], X_train_oh.shape[2]))
a0 = Input(shape=(X_train_oh.shape[1],), name='a0')
c0 = Input(shape=(X_train_oh.shape[1],), name='c0')
a = a0
c = c0
outputs=[]
for t in range(X_train_oh.shape[1]):
        
    # Step 2.A: select the "t"th time step vector from X. 
    x = Lambda(lambda x: X[:,t,:])(X)
    # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
    x = reshapor(x)
    x = dropout(x)
    # Step 2.C: Perform one step of the LSTM_cell
    a, _, c = LSTM_cell(x, initial_state=[a0, c0])
    # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
    outputs.append(a)
#out = densor(outputs)
output = Concatenate()(outputs)
output = Reshape([-1])(output)
output = dropout(output)
out = densor(output)
model = Model(inputs=[X,a0,c0],outputs=out)
opt = Adam(lr=0.07, decay=0.1475)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
a0_tr = np.zeros((X_train_oh.shape[0], X_train_oh.shape[1]))
c0_tr = np.zeros((X_train_oh.shape[0], X_train_oh.shape[1]))
a0_val = np.zeros((X_val_oh.shape[0], X_val_oh.shape[1]))
c0_val = np.zeros((X_val_oh.shape[0], X_val_oh.shape[1]))
history = model.fit([X_train_oh, a0_tr, c0_tr], Y_train, epochs=20, validation_data=([X_val_oh, a0_val, c0_val], Y_val),batch_size=500)

Train on 3500 samples, validate on 500 samples
Epoch 1/20
3500/3500 [==============================] - 54s 15ms/step - loss: 1.0482 - binary_accuracy: 0.5114 - val_loss: 0.6555 - val_binary_accuracy: 0.5720
Epoch 2/20
3500/3500 [==============================] - 16s 5ms/step - loss: 0.6586 - binary_accuracy: 0.5654 - val_loss: 0.5937 - val_binary_accuracy: 0.7540
Epoch 3/20
3500/3500 [==============================] - 17s 5ms/step - loss: 0.5220 - binary_accuracy: 0.7260 - val_loss: 0.5441 - val_binary_accuracy: 0.7060
Epoch 4/20
3500/3500 [==============================] - 16s 5ms/step - loss: 0.4391 - binary_accuracy: 0.8194 - val_loss: 0.5023 - val_binary_accuracy: 0.7720
Epoch 5/20
3500/3500 [==============================] - 16s 5ms/step - loss: 0.3875 - binary_accuracy: 0.8531 - val_loss: 0.4664 - val_binary_accuracy: 0.7880
Epoch 6/20
3500/3500 [==============================] - 16s 4ms/step - loss: 0.3388 - binary_accuracy: 0.8711 - val_loss: 0.4409 - val_binary_accuracy: 0.826

In [ ]:
a01 = np.zeros((500, X_train_oh.shape[1]))
c01 = np.zeros((500, X_train_oh.shape[1]))
scores = model.evaluate([X_test_oh, a01, c01], Y_test, verbose=0)
print(scores)

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.plot(history.history['binary_accuracy'])
pyplot.plot(history.history['val_binary_accuracy'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()